In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import joblib

sys.path.insert(0, "..")   

from src.ML_benchmark_prediction_analysis import (
    metrics_per_category
)

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
path = Path("../output/data/data_km_with_features.joblib")
df = joblib.load(path)
df.head()

,ECNumber,Type,Organism,Substrate,Kcat,Km,pH,Temp,Smiles,Sequence,log10_Kcat,log10_Km,Kcat_Unit,Km_Unit,metabolite_features,protein_features
0,1.1.99.31,mutant,Pseudomonas putida,2-Hydroxy-3-butynoate,14.800000,4.30,7.5,20.0,C#CC(C(=O)[O-])O,MSQNLFNVEDYRKLRQKRLPKMVYDYLEGGAEDEYGVKHNRDVFQQ...,1.170262,0.633468,s^(-1),mM,"[0.1228977, 0.7496512, -0.24066311, -0.4157082...","[-0.016876074, -0.05564796, 0.076313846, 0.080..."
1,1.1.99.31,wildtype,Pseudomonas putida,2-Hydroxy-3-butynoate,3.900000,22.00,7.5,20.0,C#CC(C(=O)[O-])O,MSQNLFNVEDYRKLRQKRLPKMVYDYLEGGAEDEYGVKHNRDVFQQ...,0.591065,1.342423,s^(-1),mM,"[0.1228977, 0.7496512, -0.24066311, -0.4157082...","[-0.016387729, -0.055583365, 0.07538496, 0.082..."
2,1.4.3.2,wildtype,Crotalus atrox,L-propargylglycine,7.750000,11.78,7.5,28.0,C#CCC(C(=O)O)N,MNVFFMFSLLFLAALGSCAHDRNPLEECFRETDYEEFLEIAKNGLT...,0.889302,1.071145,s^(-1),mM,"[0.08531234, 0.018572615, 0.079737425, -0.7695...","[-0.0658514, -0.14893617, 0.073952675, 0.22143..."
3,1.1.1.169,mutant,Escherichia coli,Propargylamine,NaN,65.00,7.5,25.0,C#CCN,MKITVLGCGALGQLWLTALCKQGHEVQGWLRVPQPYCSVNLVETDG...,NaN,1.812913,NaN,mM,"[-0.061515212, 0.5467514, 0.27831262, -1.00274...","[-0.013440645, -0.18082525, 0.061067406, -0.07..."
4,1.6.5.2,wildtype,Kluyveromyces marxianus,"2,3-Dichloro-5,6-dicyano-1,4-benzoquinone",8.666667,298.20,6.5,25.0,C(#N)C1=C(C(=O)C(=C(C1=O)Cl)Cl)C#N,MSSFLSKRFISTTQRAMSQLPKAKSLIYSSHDQDVSKILKVHTYQP...,0.937852,2.474508,s^(-1),mM,"[0.30274612, 0.08312379, 0.14933385, -0.178018...","[-0.066582374, 0.018061409, 0.21740742, 0.0066..."


In [5]:
df["Organism"].unique()

array(['Pseudomonas putida', 'Crotalus atrox', 'Escherichia coli', ...,
       'Ideonella dechloratans', 'Acidianus ambivalens',
       'Starkeya novella'], dtype=object)

In [4]:
df.columns

Index(['Gene', 'Role', 'ICD10_Code', 'UMLS', 'Disease_Type',
       'Association_Type', 'Disease_Name', 'Entry', 'Reviewed', 'Entry Name_x',
       'Protein names', 'Gene Names_x', 'Organism', 'Length', 'Sequence',
       'EC number', 'Active site', 'Binding site', 'Rhea ID', 'Pathway',
       'Catalytic activity', 'Tissue specificity', 'Uniprot', 'Gene_x',
       'Ensembl', 'Target Class', 'FDA Approved', 'Entry Name_y',
       'Gene Names_y', 'Gene_y', 'component_id', 'chembl_id',
       'target_pref_name', 'protein_class_name', 'class_level', 'Level_1',
       'Level_2', 'Level_3', 'Level_4', 'Level_5', 'Level_6', 'Level_7',
       'accession', 'sequence', 'organism'],
      dtype='object')

In [5]:
df["Role"].unique()

array(['TSG', 'Oncogene', 'Both'], dtype=object)

In [27]:
features = ["Gene", "Role", "Sequence"]
df = df[features]
df.rename(columns={'Sequence': 'Target'}, inplace=True)
df.head()

,Gene,Role,Target
0,SH2B3,TSG,MNGPALQPSSPSSAPSASPAAAPRGWSEFCELHAVAAARELARQYW...
1,STK11,TSG,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...
2,STK11,TSG,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...
3,STK11,TSG,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...
4,STK11,TSG,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...


In [31]:
df.dtypes

Gene      object
Role      object
Target    object
dtype: object

In [19]:
import pyarrow.parquet as pq
path = Path("../output/data/combined_predictions_BindingDB.parquet")
df2 = pd.read_parquet(path,)
# Memory efficient approach for large files
# parquet_file = pq.ParquetFile(path)
# first_batch = next(parquet_file.iter_batches(batch_size=1000))
# df = first_batch.to_pandas()


In [20]:
df2.head()

,row_index,Drug,BindingDB Ligand Name,Target Name,Target,Mutant,Kd (nM),Affinity,EC number,Target Class,...,Target_Class_Level_4,Target_Class_Level_5,Target_Class_Level_6,Drug_Features,Target_Features,Model,pred_affinity,Dataset,Split mode,Embedding
0,0,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,CHEMBL4209679,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,Mutant,27000.0,4.568636,None,None,...,None,None,None,"[0.4142605662345886, 0.4228456914424896, 0.222...","[-0.024846425279974937, 0.31483280658721924, -...",GBM,5.712360,BindingDB,random,ESMv1
1,0,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,CHEMBL4209679,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,Mutant,27000.0,4.568636,None,None,...,None,None,None,"[0.4142605662345886, 0.4228456914424896, 0.222...","[-0.024846425279974937, 0.31483280658721924, -...",XGBoost,5.183766,BindingDB,random,ESMv1
2,1,OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Mutant,10000.0,5.000000,2.7.11.1,Enzyme,...,AGC protein kinase group,AGC protein kinase RSK family,AGC protein kinase RSK subfamily,"[0.603754997253418, 0.49803397059440613, 0.045...","[-0.13943365216255188, -0.04169353097677231, -...",GBM,4.998585,BindingDB,random,ESMv1
3,1,OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Mutant,10000.0,5.000000,2.7.11.1,Enzyme,...,AGC protein kinase group,AGC protein kinase RSK family,AGC protein kinase RSK subfamily,"[0.603754997253418, 0.49803397059440613, 0.045...","[-0.13943365216255188, -0.04169353097677231, -...",XGBoost,4.958881,BindingDB,random,ESMv1
4,2,CSc1cnc(N[C@H]2CC[C@@H](C2)Nc2ccc(cn2)-c2ccccc...,"US11248001, Example 467",Proprotein convertase subtilisin/kexin type 9,MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALR...,Mutant,200.0,6.698970,3.4.21.-,Enzyme,...,Serine protease SB clan,Serine protease S8A subfamily,None,"[0.7361834645271301, 0.798686683177948, 0.7603...","[-0.02277339994907379, 0.23574493825435638, -0...",GBM,5.742586,BindingDB,random,ESMv1


In [21]:
df2.columns

Index(['row_index', 'Drug', 'BindingDB Ligand Name', 'Target Name', 'Target',
       'Mutant', 'Kd (nM)', 'Affinity', 'EC number', 'Target Class',
       'FDA Approved', 'Target_Class_Level_1', 'Target_Class_Level_2',
       'Target_Class_Level_3', 'Target_Class_Level_4', 'Target_Class_Level_5',
       'Target_Class_Level_6', 'Drug_Features', 'Target_Features', 'Model',
       'pred_affinity', 'Dataset', 'Split mode', 'Embedding'],
      dtype='object')

In [22]:
features = ['BindingDB Ligand Name',
            'Drug',
            #'Uniprot',
            'Target Name',
            'Target',
            'Target Class',
            'Target_Class_Level_1', 
            'Target_Class_Level_2',
            'Target_Class_Level_3',
            'Mutant',
            'Affinity',
            'pred_affinity',
            'Embedding',
           'Split mode',
]
df2 = df2[features]
df2.rename(columns={'BindingDB Ligand Name': 'Drug_Name',
                    'Affinity': 'True_Affinity',
                    'Embedding': 'Model',
                    'Target Class': 'Target_Class',
                    'pred_affinity': 'Predicted_Affinity', 
                    'Split mode': 'Dataset',
                    }, inplace=True)

In [23]:
df2.head()

,Drug_Name,Drug,Target Name,Target,Target_Class,Target_Class_Level_1,Target_Class_Level_2,Target_Class_Level_3,Mutant,True_Affinity,Predicted_Affinity,Model,Dataset
0,CHEMBL4209679,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,None,None,None,None,Mutant,4.568636,5.712360,ESMv1,random
1,CHEMBL4209679,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,None,None,None,None,Mutant,4.568636,5.183766,ESMv1,random
2,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Enzyme,Enzyme,Kinase,Protein Kinase,Mutant,5.000000,4.998585,ESMv1,random
3,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Enzyme,Enzyme,Kinase,Protein Kinase,Mutant,5.000000,4.958881,ESMv1,random
4,"US11248001, Example 467",CSc1cnc(N[C@H]2CC[C@@H](C2)Nc2ccc(cn2)-c2ccccc...,Proprotein convertase subtilisin/kexin type 9,MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALR...,Enzyme,Enzyme,Protease,Serine protease,Mutant,6.698970,5.742586,ESMv1,random


In [18]:
df2["Mutant"].unique()

array(['Mutant', 'No match', 'Wildtype'], dtype=object)

In [30]:
df2.dtypes

Drug_Name                object
Drug                     object
Target Name              object
Target                   object
Target_Class             object
Target_Class_Level_1     object
Target_Class_Level_2     object
Target_Class_Level_3     object
Mutant                   object
True_Affinity           float64
Predicted_Affinity      float64
Model                    object
Dataset                  object
dtype: object

In [33]:
df.dtypes

Gene      object
Role      object
Target    object
dtype: object

In [35]:
df3 = pd.merge(df2, df, on="Target", how='left')

In [36]:
df3.head()

,Drug_Name,Drug,Target Name,Target,Target_Class,Target_Class_Level_1,Target_Class_Level_2,Target_Class_Level_3,Mutant,True_Affinity,Predicted_Affinity,Model,Dataset,Gene,Role
0,CHEMBL4209679,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,None,None,None,None,Mutant,4.568636,5.712360,ESMv1,random,NaN,NaN
1,CHEMBL4209679,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,None,None,None,None,Mutant,4.568636,5.183766,ESMv1,random,NaN,NaN
2,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Enzyme,Enzyme,Kinase,Protein Kinase,Mutant,5.000000,4.998585,ESMv1,random,NaN,NaN
3,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Enzyme,Enzyme,Kinase,Protein Kinase,Mutant,5.000000,4.958881,ESMv1,random,NaN,NaN
4,"US11248001, Example 467",CSc1cnc(N[C@H]2CC[C@@H](C2)Nc2ccc(cn2)-c2ccccc...,Proprotein convertase subtilisin/kexin type 9,MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALR...,Enzyme,Enzyme,Protease,Serine protease,Mutant,6.698970,5.742586,ESMv1,random,NaN,NaN


In [38]:
df3["Gene"].unique()

array([nan, 'CSF1R', 'CRBN', 'MTOR', 'MET', 'PIK3CB', 'ABL1', 'FGFR3',
       'FGFR4', 'FLT3', 'MAP2K1', 'BTK', 'ERBB2', 'EZH2', 'KEAP1',
       'FGFR2', 'CBLB', 'NFE2L2', 'EPHA2', 'FLT4', 'CBLC', 'BRAF',
       'PRKCB', 'STK11', 'FGFR1', 'CHEK1', 'INSR', 'BCL6', 'PIK3CD',
       'KIT', 'JAK3', 'MAP2K2', 'TYK2', 'PDGFRB', 'SMARCA2', 'CREBBP',
       'BMPR1A', 'MAP2K4', 'JAK2', 'RPS6KA3', 'EGFR', 'IKBKB', 'RAF1',
       'SOS1', 'PPARG', 'KDR', 'CHEK2', 'NTRK1', 'AKT3', 'ALK', 'XIAP',
       'EP300', 'MAP3K7', 'EPAS1', 'PIK3CA', 'RET', 'ACVR1', 'DDR2',
       'AKT1', 'MTAP', 'KRAS', 'IDH1', 'AR', 'CDK8', 'MERTK', 'EPHB1',
       'EPHB4', 'PDGFRA', 'NTRK2', 'ROS1', 'ERBB4', 'PAK3', 'ACVR1B',
       'AKT2', 'IGF1R'], dtype=object)

In [41]:
df3[:1000].to_csv("../output/data/combined_predictions_BindingDB_with_gene.csv", index=False)

In [33]:
res.to_csv("../output/data/prediction_target_class_analysis.csv", index=False)

In [35]:
top_k = 10
min_n = 5
res = metrics_per_category(
            df,
            "Target Class",
            by=["Embedding"],
            top_k=top_k,
            min_n=min_n,
)
res.head()

/Users/yvesgreatti/github/drug_discovery/src/ML_benchmark_prediction_analysis.py:148: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ]


,Embedding,category,n,rmse,mae,mse,r2,pearson,median_ae,explained_variance
0,ProteinCLIP,Enzyme,2954.0,1.217115,0.911841,1.481370,0.173177,0.463002,0.706449,0.184095
1,ESM2,Enzyme,2954.0,1.226950,0.922839,1.505407,0.159761,0.455586,0.727113,0.171109
2,MUTAPLM,Enzyme,2954.0,1.253469,0.942858,1.571185,0.123047,0.436746,0.728664,0.142039
3,ESMv1,Enzyme,2954.0,1.272035,0.947503,1.618073,0.096876,0.426688,0.722131,0.097906
4,ESM2,GPCR,444.0,1.095626,0.874980,1.200397,-0.156156,0.347409,0.739559,0.053968


In [36]:
res.to_csv("../output/data/prediction_embedding_target_class_analysis.csv", index=False)

In [38]:
top_k = 10
min_n = 5
res = metrics_per_category(
            df,
            "Mutant",
            by=["Embedding"],
            top_k=top_k,
            min_n=min_n,
)
res.head()

/Users/yvesgreatti/github/drug_discovery/src/ML_benchmark_prediction_analysis.py:148: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ]


,Embedding,category,n,rmse,mae,mse,r2,pearson,median_ae,explained_variance
0,ESM2,Mutant,3288.0,1.257348,0.961553,1.580923,0.210568,0.496133,0.771457,0.227334
1,MUTAPLM,Mutant,3288.0,1.294957,0.993061,1.676913,0.162636,0.470563,0.797684,0.188594
2,ESMv1,Mutant,3288.0,1.383590,1.062740,1.914322,0.044086,0.444252,0.834271,0.066152
3,ProteinCLIP,Mutant,3288.0,1.397452,1.065936,1.952872,0.024836,0.397982,0.834713,0.081640
4,ESM2,No match,1778.0,1.398066,1.056359,1.954588,0.277872,0.551784,0.829176,0.289326


In [39]:
res.to_csv("../output/data/prediction_embedding_mutant_analysis.csv", index=False)

In [ ]:
# Create a simple scatter plot of predictions vs true values
plt.figure(figsize=(10, 8))

# Plot by embedding type
embeddings = df['Embedding'].unique()
colors = plt.cm.Set1(np.linspace(0, 1, len(embeddings)))

for i, embedding in enumerate(embeddings):
    mask = df['Embedding'] == embedding
    plt.scatter(df[mask]['Affinity'], df[mask]['affinity_pred'], 
               alpha=0.6, label=embedding, color=colors[i], s=20)

# Add perfect prediction line
min_val = min(df['Affinity'].min(), df['affinity_pred'].min())
max_val = max(df['Affinity'].max(), df['affinity_pred'].max())
plt.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.8, label='Perfect Prediction')

plt.xlabel('True Affinity', fontsize=12)
plt.ylabel('Predicted Affinity', fontsize=12)
plt.title('Predicted vs True Affinity Values (1000 samples)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [4]:
df = pd.read_parquet("../output/metrics/20251031_all_binding_db_genes_small.parquet")
cols = ['Drug_Name',
 'Model',
 'Gene',
 'Role',
 'Dataset',
 'Drug',
 'Target Name',
 'Target',
 'Mutant',
 'Target_Class',
 'Target_Class_Level_1',
 'Target_Class_Level_2',
 'Target_Class_Level_3',
 'True_Affinity',
 'Predicted_Affinity',
]
df = df[cols]   
df.head()

,Drug_Name,Model,Gene,Role,Dataset,Drug,Target Name,Target,Mutant,Target_Class,Target_Class_Level_1,Target_Class_Level_2,Target_Class_Level_3,True_Affinity,Predicted_Affinity
0,CHEMBL4209679,ESMv1,None,None,random,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,Mutant,None,None,None,None,4.568636,5.089439
1,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",ESMv1,None,None,random,OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Mutant,Enzyme,Enzyme,Kinase,Protein Kinase,5.000000,5.154931
2,"US11248001, Example 467",ESMv1,None,None,random,CSc1cnc(N[C@H]2CC[C@@H](C2)Nc2ccc(cn2)-c2ccccc...,Proprotein convertase subtilisin/kexin type 9,MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALR...,Mutant,Enzyme,Enzyme,Protease,Serine protease,6.698970,6.506505
3,AZD-6244::CHEMBL1614701,ESMv1,None,None,random,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(cc12)C(=O)NOCCO,Myosin-IIIa,MFPLIGKTIIFDNFPDPSDTWEITETIGKGTYGKVFKVLNKKNGQK...,No match,None,None,None,None,5.000000,5.000000
4,PKC-412::cid_24202429,ESMv1,CSF1R,Oncogene,random,CO[C@H]1[C@H](C[C@@H]2O[C@@]1(C)n1c3ccccc3c3c4...,Macrophage colony-stimulating factor 1 receptor,MGPGVLLLLLVATAWHGQGIPVIEPSVPELVVKPGATVTLRCVGNG...,Mutant,Enzyme,Enzyme,Kinase,Protein Kinase,6.481486,6.542833


In [3]:
df.columns.tolist()

['Drug_Name',
 'Drug',
 'Target Name',
 'Target',
 'Target_Class',
 'Target_Class_Level_1',
 'Target_Class_Level_2',
 'Target_Class_Level_3',
 'Mutant',
 'True_Affinity',
 'Predicted_Affinity',
 'Model',
 'Dataset',
 'Gene',
 'Role']

In [9]:
from scipy.stats import pearsonr
def _one_group(g: pd.DataFrame, y_col: str, yhat_col: str) -> pd.Series:
    """Calculate metrics for one group, handling small sample sizes"""
    y_true = g[y_col].astype(float).to_numpy()
    y_pred = g[yhat_col].astype(float).to_numpy()

    # Check if we have enough data points
    if len(y_true) < 2:
        return pd.Series(
            {
                "n": len(g),
                "rmse": np.nan,
                "mae": np.nan,
                "mse": np.nan,
                "r2": np.nan,
                "pearson": np.nan,
                "median_ae": np.nan,
                "explained_variance": np.nan,
            }
        )

    try:
        # Calculate metrics safely
        mse = np.mean((y_true - y_pred) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(y_true - y_pred))
        median_ae = np.median(np.abs(y_true - y_pred))

        # Handle R2 calculation
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else np.nan

        # Handle correlation calculation
        if len(np.unique(y_true)) > 1 and len(np.unique(y_pred)) > 1:
            pearson_corr, _ = pearsonr(y_true, y_pred)
        else:
            pearson_corr = np.nan

        # Explained variance
        var_y = np.var(y_true)
        explained_variance = (
            1 - np.var(y_true - y_pred) / var_y if var_y != 0 else np.nan
        )

        return pd.Series(
            {
                "n": len(g),
                "rmse": rmse,
                "mae": mae,
                "mse": mse,
                "r2": r2,
                "pearson": pearson_corr,
                "median_ae": median_ae,
                "explained_variance": explained_variance,
            }
        )

    except Exception as e:
        print(f"Error calculating metrics for group of size {len(g)}: {e}")
        return pd.Series(
            {
                "n": len(g),
                "rmse": np.nan,
                "mae": np.nan,
                "mse": np.nan,
                "r2": np.nan,
                "pearson": np.nan,
                "median_ae": np.nan,
                "explained_variance": np.nan,
            }
        )


In [11]:
#    result = (
#             dfe.groupby(group_cols, dropna=False)
#             .apply(lambda g: _one_group(g, y_col, yhat_col), include_groups=False)
#             .reset_index()
#         )
df.groupby(["Model", "Mutant"], dropna=False)\
    .apply(lambda g: _one_group(g, "True_Affinity", "Predicted_Affinity"), include_groups=True).reset_index()

/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_16282/2263375477.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: _one_group(g, "True_Affinity", "Predicted_Affinity"), include_groups=True).reset_index()


,Model,Mutant,n,rmse,mae,mse,r2,pearson,median_ae,explained_variance
0,ESMv1,Mutant,778.0,1.593726,1.571661,2.539961,-8.984939,0.967363,1.618957,0.603585
1,ESMv1,No match,221.0,1.236179,1.226835,1.528138,-16.988912,0.749482,1.248806,0.368008
2,ESMv1,Wildtype,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.groupby(["Model", ""], dropna=False).apply(lambda g: , include_groups=True).reset_index()